In [46]:
import graphlab as gl
import pickle
import pandas as pd
import numpy as np
from collections import Counter

In [47]:
data_items = pickle.load(open("/Users/marvinbertin/Github/family_style_chat_bot/data/user_by_cuisine_by_dish_ratings.pkl", 'rb'))
data_cuisine = pickle.load(open("/Users/marvinbertin/Github/family_style_chat_bot/data/user_by_cuisine_ratings.pkl", 'rb'))

In [48]:
df_cuisine = pd.DataFrame(data_cuisine)
df_cuisine.head()

,user_id,item_id,rating
0,Brian,Lime Tree Southeast Asian Kitchen,2
1,David,Lime Tree Southeast Asian Kitchen,0.5
3,Jon,Lime Tree Southeast Asian Kitchen,4
4,Marvin,Lime Tree Southeast Asian Kitchen,0.5
6,Anne,Lime Tree Southeast Asian Kitchen,2.5


In [49]:
df_cuisine.pivot(index='user_id', columns="item_id")

rating                                            \
item_id  Chapeau! Jayakarta Supreme La Urbana Lers Ros Thai   
user_id                                                       
Anne         None                 4       1.5             4   
Bob          None              None      None           2.5   
Brian        None              None       3.5             5   
David           5              None      None           1.5   
Eugene        0.5              None      None             2   
Jon           2.5                 4         1             5   
Leah         None                 1      None           1.5   
Marvin        4.5               4.5      None          None   
Michelle        5              None         1          None   
Samatha      None                 5         3           3.5   
Sarah        None               1.5      None             3   

                                                                           \
item_id  Lime Tree Southeast Asian Kitchen Little Delhi Pizza Saigon Cafe   
user_id                                                                     
Anne                                   2.5            5     5           4   
Bob                                   None          3.5   1.5           2   
Brian                                    2         None  None           5   
David                                  0.5         None     2         1.5   
Eugene                                None            1  None        None   
Jon                                      4         None   2.5           3   
Leah                                     4            5  None        None   
Marvin                                 0.5            2  None        None   
Michelle                               4.5          1.5   1.5        None   
Samatha                                  2         None   2.5         2.5   
Sarah                                  0.5            1   2.5        None   

                                                                   
item_id  Sanraku Shanghai Dumplings Tommaso's Ristorante Italiano  
user_id                                                            
Anne         1.5                  4                          None  
Bob            3                2.5                          None  
Brian       None                4.5                           3.5  
David       None                3.5                           3.5  
Eugene      None                  2                             1  
Jon         None                0.5                          None  
Leah         2.5               None                          None  
Marvin      None               None                           1.5  
Michelle    None                4.5                          None  
Samatha     None               None                           4.5  
Sarah       None               None                             4

In [50]:
data_items.keys()

['Lers Ros Thai',
 'Shanghai Dumplings',
 'Saigon Cafe',
 'Little Delhi',
 'Sanraku',
 'La Urbana',
 'Jayakarta Supreme',
 'Chapeau!',
 'Lime Tree Southeast Asian Kitchen',
 "Tommaso's Ristorante Italiano",
 'Pizza']

In [51]:
class group_recommender(object):
    def __init__(self, cuisine_sf, dict_cuisine_items):
        self.cuisine = gl.SFrame(cuisine_sf)
        self.cuisine_items = dict_cuisine_items
    
    def recommend(self, group_list):
        group_name = "_".join(group_list)
        
        sf_avg_user = self.cuisine.filter_by(group_list, "user_id") \
                                  .groupby(key_columns='item_id',
                                           operations={'rating': gl.aggregate.MEAN('rating')})  
        
        sf_avg_user.add_column(gl.SArray([group_name] * len(sf_avg_user)), "user_id")
#         print sf_avg_user
        
        sf_new = self.cuisine.append(sf_avg_user)
        
        model = gl.recommender.create(sf_new, target='rating')
    
        results = model.recommend([group_name], exclude_known=False)
#         print results
        result_cuisine = results["item_id"][:3]
    
        option_list = []
        for cuisine in result_cuisine:
            
            sf_items = gl.SFrame(self.cuisine_items[cuisine])
            model_items = gl.recommender.create(sf_items, target='rating')

            results_items = model_items.recommend(group_list, exclude_known=False, k = 2)
    #         print results_items
                
            if cuisine == "Pizza":
                group_size = len(group_list)
                num_pizza = int(group_size / 1.5)
                item_results = [item for item, count in Counter(results_items["item_id"]).most_common()][:num_pizza]
                option_list.append(("Pizza Party!", item_results))
        
            else:
                group_size = len(group_list)
                item_results = [item for item, count in Counter(results_items["item_id"]).most_common()][:group_size]
                option_list.append((cuisine, item_results))

        return option_list

In [56]:
group_list = np.random.choice(df_cuisine["user_id"].unique(), size = 2, replace=False)

print group_list
model = group_recommender(df_cuisine, data_items)
result = model.recommend(group_list)

['Anne' 'Brian']


Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 80 observations with 12 users and 11 items.

Data prepared in: 0.007774s

...

| Initial | 72us         | 4.43158           | 1.49204               |             |

+---------+--------------+-------------------+-----------------------+-------------+

| 1       | 775us        | 5.2988            | 1.95047               | 0.390625    |

| 2       | 1.722ms      | 4.81267           | 1.842                 | 0.232267    |

| 3       | 2.446ms      | 3.76301           | 1.58725               | 0.171364    |

In [57]:
result

[('Lers Ros Thai', ['Thai Ginger Soup', 'Fried Chicken Wings']),
 ('Jayakarta Supreme', ['Indonesian Spiced Rice', 'Indonesia Satay']),
 ('Shanghai Dumplings', ['Chinese Mabo Tofu', 'Chinese Tea Leaf Eggs'])]